In [165]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
import re
import unidecode
from tqdm import tqdm
tqdm.pandas()

pd.options.display.max_columns=None

In [253]:
pos = pd.read_csv('data/raw_from_scrape/players_position.csv')
players_sofifa = pd.read_csv('data/players_sofifa.csv')


#transformation de la position

In [254]:
# Extract horizontal and vertical position on the field
pos['horizontal'] = pos.field_pos.progress_apply(lambda x: np.float16(re.findall(r'\d+\.*\d*%',x)[0].replace('%','')))
pos['vertical'] = pos.field_pos.apply(lambda x: np.float16(re.findall(r'\d+\.*\d*%',x)[1].replace('%','')))
pos.drop(columns = ['field_pos'], inplace = True)

100%|██████████| 220/220 [00:00<00:00, 79912.26it/s]


In [255]:
def search_position(x,to_search):
    try:
       res = players_sofifa.loc[(players_sofifa.club_id == x['club_id']) & (players_sofifa.number == x['number']) ,to_search].values[0] 
    except:
        try:
            res = players_sofifa.loc[(players_sofifa.club_id == x['club_id']) & (players_sofifa.short_name == x['name']) ,to_search].values[0]
        except:
            res = None
    return res



In [256]:
players_sofifa

,club,club_id,number,name,short_name,DoB,poste,lequipe_id,sofifa_id
0,Angers,374,16.0,Anthony Mandrea,A. Mandrea,1996-12-25,Gar.,50523,220833.0
1,Angers,374,30.0,Danijel Petkovic,D. Petkovic,1993-05-25,Gar.,53178,240212.0
2,Angers,374,25.0,Abdoulaye Bamba,A. Bamba,1990-04-25,Déf.,34789,199313.0
3,Angers,374,20.0,Kevin Boma,K. Boma,2002-11-20,Déf.,77429,258457.0
4,Angers,374,NaN,Elhadji Pape Diaw,E. Diaw,1994-12-31,Déf.,71974,NaN
...,...,...,...,...,...,...,...,...,...
710,Bordeaux,18,11.0,Sekou Mara,S. Mara,2002-07-30,Att.,82623,257612.0
711,Bordeaux,18,23.0,M'Baye Niang,M. Niang,1994-12-19,Att.,37439,204077.0
712,Bordeaux,18,NaN,Moudja Sie Ouattara,M. Ouattara,2002-05-20,Att.,82622,NaN
713,Bordeaux,18,28.0,Rémi Oudin,R. Oudin,1996-11-18,Att.,59643,233265.0


In [257]:
# seek for player id and sofifa_id from l'equipe.fr
pos['player_id'] = pos.progress_apply(lambda x: search_position(x,'lequipe_id')\
    , axis = 1)
pos['sofifa_id'] = pos.progress_apply(lambda x: search_position(x,'sofifa_id')\
    , axis = 1)
pos['poste'] = pos.progress_apply(lambda x: search_position(x,'poste')\
, axis = 1)



100%|██████████| 220/220 [00:00<00:00, 1364.22it/s]


In [258]:

display(pos.loc[pos.player_id.isnull(),:])
display(pos.loc[pos.sofifa_id.isnull(),:])
display(pos.loc[pos.poste.isnull(),:])



,journee,game_id,club_id,number,name,horizontal,vertical,player_id,sofifa_id,poste


,journee,game_id,club_id,number,name,horizontal,vertical,player_id,sofifa_id,poste
16,22,515694,38,33,A. Bakayoko,73.75,80.625,86331,NaN,Déf.


,journee,game_id,club_id,number,name,horizontal,vertical,player_id,sofifa_id,poste


In [259]:

pos.loc[:,['journee','game_id','club_id','player_id', 'sofifa_id', 'poste', 'horizontal','vertical']].to_csv('data/players_positions.csv' ,index = False)

In [260]:
pos = pd.read_csv('data/players_positions.csv')

In [228]:
stat = np.asarray(['overall' ,'potential', 
'team', 'nat_team', 'best_overall_rating', 
'attacking_crossing', 'attacking_finishing', 'attacking_heading_accuracy', 'attacking_short_passing', 'attacking_volleys',
'skill_dribbling','skill_curve','skill_fk_accuracy','skill_long_passing','skill_ball_control', 
'movement_acceleration', 'movement_sprint_speed', 'movement_agility', 'movement_reactions', 'movement_balance', 
'power_shot_power', 'power_jumping', 'power_stamina', 'power_strength', 'power_long_shots',
'mentality_aggression', 'mentality_interceptions', 'mentality_positioning', 'mentality_vision', 'mentality_penalties', 'mentality_composure', 
'defending_marking_awareness', 'defending_standing_tackle', 'defending_sliding_tackle',
'goalkeeping_diving','goalkeeping_handling','goalkeeping_kicking','goalkeeping_positioning','goalkeeping_reflexes','goalkeeping_speed'])